In [26]:
#r "nuget: Jinaga, 0.12.5"
#r "nuget: Jinaga.Notebooks, 0.12.5"

Installed Packages Jinaga, 0.12.5 Jinaga.Notebooks, 0.12.5

In [27]:
using Jinaga;
using Jinaga.Http;
using Jinaga.Notebooks;
using Jinaga.Notebooks.Authentication;

Create a file called `settings.json` and put the following settings into it:

```json
{
    "HttpEndpoint": "https://rep.jinaga.com/....",
    "AuthorizationToken": "eyJh...."
}
```

In [28]:
#!value --from-file settings.json --name settings

In [29]:
#!share settings --from value

var parsedSettings = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, string>>(settings);

var jinagaClient = JinagaClient.Create(options =>
{
    options.HttpEndpoint = new Uri(parsedSettings["HttpEndpoint"]);
    options.HttpAuthenticationProvider = new TokenAuthenticationProvider(parsedSettings["AuthorizationToken"]);
});

In [30]:
[FactType("Blog.User.Name")]
public record UserName(User user, string value, UserName[] prior) { }

[FactType("Blog.Site")]
public record Site(User creator, DateTime createdAt) { }

[FactType("Blog.Site.Deleted")]
public record SiteDeleted(Site site, DateTime deletedAt) { }

[FactType("Blog.Site.Restored")]
public record SiteRestored(SiteDeleted deleted) { }

[FactType("Blog.Site.Name")]
public record SiteName(Site site, string value, SiteName[] prior) { }

[FactType("Blog.Site.Domain")]
public record SiteDomain(Site site, string value, SiteDomain[] prior) { }

[FactType("Blog.Post")]
public record Post(Site site, User author, DateTime createdAt) { }

[FactType("Blog.Post.Title")]
public record PostTitle(Post post, string value, PostTitle[] prior) { }

[FactType("Blog.Post.Deleted")]
public record PostDeleted(Post post, DateTime deletedAt) { }

[FactType("Blog.Post.Restored")]
public record PostRestored(PostDeleted deleted) { }

[FactType("Blog.Post.Publish")]
public record Publish(Post post, DateTime date) { }

In [31]:
Renderer.RenderTypes(typeof(Site), typeof(SiteName), typeof(SiteDomain), typeof(SiteDeleted), typeof(SiteRestored))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator 
 
<!-- Blog.Site.Name -->
 
 Blog.Site.Name 
 
 Blog.Site.Name 
 
<!-- Blog.Site.Name->Blog.Site -->
 
 Blog.Site.Name->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Name->Blog.Site.Name -->
 
 Blog.Site.Name->Blog.Site.Name 
 
 
 prior 
 
<!-- Blog.Site.Domain -->
 
 Blog.Site.Domain 
 
 Blog.Site.Domain 
 
<!-- Blog.Site.Domain->Blog.Site -->
 
 Blog.Site.Domain->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Domain->Blog.Site.Domain -->
 
 Blog.Site.Domain->Blog.Site.Domain 
 
 
 prior 
 
<!-- Blog.Site.Deleted -->
 
 Blog.Site.Deleted 
 
 Blog.Site.Deleted 
 
<!-- Blog.Site.Deleted->Blog.Site -->
 
 Blog.Site.Deleted->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Restored -->
 
 Blog.Site.Restored 
 
 Blog.Site.Restored 
 
<!-- Blog.Site.Restored->Blog.Site.Deleted -->
 
 Blog.Site.Restored->Blog.Site.Deleted 
 
 
 deleted

In [32]:
var sitesForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    select new
    {
        id = jinagaClient.Hash(site),
        names =
            from name in facts.OfType<SiteName>()
            where name.site == site &&
                !facts.Any<SiteName>(next => next.prior.Contains(name))
            select name.value,
        domains =
            from domain in facts.OfType<SiteDomain>()
            where domain.site == site &&
                !facts.Any<SiteDomain>(next => next.prior.Contains(domain))
            select domain.value
    }
);

(var user, var profile) = await jinagaClient.Login();
var sites = await jinagaClient.Query(sitesForUser, user);

sites

(empty)

In [33]:
user.publicKey

-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAp/JGarvueVEeYEb76CD4
8/j1tfYKx2vky0m/+EkR6yVhNBC+PwOqZ3xa+VQ2U3K+rRB2WeGB8BogR1bSZAKp
AishPVkmU0hK7mJr68PGoHQIWWKnYYSCpu/cSkkdzIoyfIgYJsTH3JPazJ0Hi34C
OXcQz4hANNlWt+7bIPJTgP7/sw/vuCQvcxnMbtmvIkvij5yS8DLQQ6AuWzlQMIjo
qr7njUtMXgXtBetmC78U55aqTLunD4iiG3MRVT7oz75frW/Qe7QZExY0YvA2A0t3
XnKhrYWDWgU1/71j3PU5tFUzua/JofsRAAKErzXw1dXhH4cLMEnuMNh1tdZcHLzI
OwIDAQAB
-----END PUBLIC KEY-----


In [34]:
user.publicKey.Substring(user.publicKey.Length - 8, 6)

Y-----

In [38]:
if (false)
{
    var michaelperrynet = await jinagaClient.Fact(new Site(user, DateTime.UtcNow));
    var michaelperrynetName = await jinagaClient.Fact(new SiteName(michaelperrynet, "Michael Perry .NET", new SiteName[0]));
    var michaelperrynetDomain = await jinagaClient.Fact(new SiteDomain(michaelperrynet, "michaelperry.net", new SiteDomain[0]));

    var qedcode = await jinagaClient.Fact(new Site(user, DateTime.UtcNow.AddMilliseconds(1)));
    var qedcodeName = await jinagaClient.Fact(new SiteName(qedcode, "QED Code", new SiteName[0]));
    var qedcodeDomain = await jinagaClient.Fact(new SiteDomain(qedcode, "qedcode.com", new SiteDomain[0]));
}

In [39]:
var siteFactsForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    select site
);

var siteNamesForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    from name in facts.OfType<SiteName>()
    where name.site == site &&
        !facts.Any<SiteName>(next => next.prior.Contains(name))
    select name
);

var siteDomainsForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    from domain in facts.OfType<SiteDomain>()
    where domain.site == site &&
        !facts.Any<SiteDomain>(next => next.prior.Contains(domain))
    select domain
);

var siteFacts = await jinagaClient.Query(siteFactsForUser, user);
var siteNames = await jinagaClient.Query(siteNamesForUser, user);
var siteDomains = await jinagaClient.Query(siteDomainsForUser, user);

jinagaClient.RenderFacts(siteFacts, siteNames, siteDomains)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- QHhAFDEW6Ky9NFm4F4Harmi5+0l1sQe2c8NAPqDbXTDCZmOEre6H8UWpSytj001CQ3WLghrbAr72kdmJ5kd+Ug== -->
 
 QHhAFDEW6Ky9NFm4F4Harmi5+0l1sQe2c8NAPqDbXTDCZmOEre6H8UWpSytj001CQ3WLghrbAr72kdmJ5kd+Ug== 
 
 Jinaga.User 
 
 publicKey 
 
 -----BEGIN PUBLIC KE... 
 
<!-- IG9sAbHdqc/5LMpeBlr5cGfZLjMPazIhIlrpu1lL46XQ62eNdaRQxKWLp1rgTl/Keg+x8ODeE9hAUNXxiqB4vg== -->
 
 IG9sAbHdqc/5LMpeBlr5cGfZLjMPazIhIlrpu1lL46XQ62eNdaRQxKWLp1rgTl/Keg+x8ODeE9hAUNXxiqB4vg== 
 
 Blog.Site 
 
 createdAt 
 
 2024-07-09T12:23:30.... 
 
 
<!-- IG9sAbHdqc/5LMpeBlr5cGfZLjMPazIhIlrpu1lL46XQ62eNdaRQxKWLp1rgTl/Keg+x8ODeE9hAUNXxiqB4vg==->QHhAFDEW6Ky9NFm4F4Harmi5+0l1sQe2c8NAPqDbXTDCZmOEre6H8UWpSytj001CQ3WLghrbAr72kdmJ5kd+Ug== -->
 
 IG9sAbHdqc/5LMpeBlr5cGfZLjMPazIhIlrpu1lL46XQ62eNdaRQxKWLp1rgTl/Keg+x8ODeE9hAUNXxiqB4vg==->QHhAFDEW6Ky9NFm4F4Harmi5+0l1sQe2c8NAPqDbXTDCZmOEre6H8UWpSytj001CQ3WLghrbAr72kdmJ5kd+Ug== 
 
 
 creator 
 
<!-- ZpBOavpusAyYEjhsE0EedYJAfb55HEyqcET58lRYb8Di98pQaZZRSt+XWamxh7ZEq/gHIkCUKHZ2GB8XxwXiIA== -->
 
 ZpBOavpusAyYEjhsE0EedYJAfb55HEyqcET58lRYb8Di98pQaZZRSt+XWamxh7ZEq/gHIkCUKHZ2GB8XxwXiIA== 
 
 Blog.Site 
 
 createdAt 
 
 2024-07-09T12:23:30.... 
 
 
<!-- ZpBOavpusAyYEjhsE0EedYJAfb55HEyqcET58lRYb8Di98pQaZZRSt+XWamxh7ZEq/gHIkCUKHZ2GB8XxwXiIA==->QHhAFDEW6Ky9NFm4F4Harmi5+0l1sQe2c8NAPqDbXTDCZmOEre6H8UWpSytj001CQ3WLghrbAr72kdmJ5kd+Ug== -->
 
 ZpBOavpusAyYEjhsE0EedYJAfb55HEyqcET58lRYb8Di98pQaZZRSt+XWamxh7ZEq/gHIkCUKHZ2GB8XxwXiIA==->QHhAFDEW6Ky9NFm4F4Harmi5+0l1sQe2c8NAPqDbXTDCZmOEre6H8UWpSytj001CQ3WLghrbAr72kdmJ5kd+Ug== 
 
 
 creator 
 
<!-- IQ+JZb0MSzmGnV7thy+nEQLtQumHRMNcM1hl9m8pMTPKn7vdlp3WshL8UfsAkPLpotrRy8Zyw6+AS+hHSXLXDw== -->
 
 IQ+JZb0MSzmGnV7thy+nEQLtQumHRMNcM1hl9m8pMTPKn7vdlp3WshL8UfsAkPLpotrRy8Zyw6+AS+hHSXLXDw== 
 
 Blog.Site.Name 
 
 value 
 
 Michael Perry .NET 
 
 
<!-- IQ+JZb0MSzmGnV7thy+nEQLtQumHRMNcM1hl9m8pMTPKn7vdlp3WshL8UfsAkPLpotrRy8Zyw6+AS+hHSXLXDw==->IG9sAbHdqc/5LMpeBlr5cGfZLjMPazIhIlrpu1lL46XQ62eNdaRQxKWLp1rgTl/Keg+x8ODeE9hAUNXxiqB4vg== -->
 
 IQ+JZb0MSzmGnV7thy+nEQLtQumHRMNcM1hl9m8pMTPKn7vdlp3WshL8UfsAkPLpotrRy8Zyw6+AS+hHSXLXDw==->IG9sAbHdqc/5LMpeBlr5cGfZLjMPazIhIlrpu1lL46XQ62eNdaRQxKWLp1rgTl/Keg+x8ODeE9hAUNXxiqB4vg== 
 
 
 site 
 
<!-- cteT8mNpUT2zthsMjGQ16k8zR6Uv9CrN627pbuAC5aYfDHI/ep9sNxgeGB8AL3DaN2HaHmHOrgJaFXtSccF4AA== -->
 
 cteT8mNpUT2zthsMjGQ16k8zR6Uv9CrN627pbuAC5aYfDHI/ep9sNxgeGB8AL3DaN2HaHmHOrgJaFXtSccF4AA== 
 
 Blog.Site.Name 
 
 value 
 
 QED Code 
 
 
<!-- cteT8mNpUT2zthsMjGQ16k8zR6Uv9CrN627pbuAC5aYfDHI/ep9sNxgeGB8AL3DaN2HaHmHOrgJaFXtSccF4AA==->ZpBOavpusAyYEjhsE0EedYJAfb55HEyqcET58lRYb8Di98pQaZZRSt+XWamxh7ZEq/gHIkCUKHZ2GB8XxwXiIA== -->
 
 cteT8mNpUT2zthsMjGQ16k8zR6Uv9CrN627pbuAC5aYfDHI/ep9sNxgeGB8AL3DaN2HaHmHOrgJaFXtSccF4AA==->ZpBOavpusAyYEjhsE0EedYJAfb55HEyqcET58lRYb8Di98pQaZZRSt+XWamxh7ZEq/gHIkCUKHZ2GB8XxwXiIA== 
 
 
 site 
 
<!-- FI/bbrFFOvqrlk5S8bvD/+j75LzOayoaox4d5KeoYrjQnBvdmCcjXpZ1SnHa5twSEUtW+Dfz4Q4621QQZDeojw== -->
 
 FI/bbrFFOvqrlk5S8bvD/+j75LzOayoaox4d5KeoYrjQnBvdmCcjXpZ1SnHa5twSEUtW+Dfz4Q4621QQZDeojw== 
 
 Blog.Site.Domain 
 
 value 
 
 michaelperry.net 
 
 
<!-- FI/bbrFFOvqrlk5S8bvD/+j75LzOayoaox4d5KeoYrjQnBvdmCcjXpZ1SnHa5twSEUtW+Dfz4Q4621QQZDeojw==->IG9sAbHdqc/5LMpeBlr5cGfZLjMPazIhIlrpu1lL46XQ62eNdaRQxKWLp1rgTl/Keg+x8ODeE9hAUNXxiqB4vg== -->
 
 FI/bbrFFOvqrlk5S8bvD/+j75LzOayoaox4d5KeoYrjQnBvdmCcjXpZ1SnHa5twSEUtW+Dfz4Q4621QQZDeojw==->IG9sAbHdqc/5LMpeBlr5cGfZLjMPazIhIlrpu1lL46XQ62eNdaRQxKWLp1rgTl/Keg+x8ODeE9hAUNXxiqB4vg== 
 
 
 site 
 
<!-- Vj2aZZ9uF/8rHfNfoLt5En9hpSYR0hcjUifHVi/kEMLHf954qxcCQh/bJe9KuOC2onUsqMTT0ORFM46t91JSyw== -->
 
 Vj2aZZ9uF/8rHfNfoLt5En9hpSYR0hcjUifHVi/kEMLHf954qxcCQh/bJe9KuOC2onUsqMTT0ORFM46t91JSyw== 
 
 Blog.Site.Domain 
 
 value 
 
 qedcode.com 
 
 
<!-- Vj2aZZ9uF/8rHfNfoLt5En9hpSYR0hcjUifHVi/kEMLHf954qxcCQh/bJe9KuOC2onUsqMTT0ORFM46t91JSyw==->ZpBOavpusAy

In [40]:
if (false)
{
    var post1a = await jinagaClient.Fact(new Post(siteFacts[0], user, DateTime.UtcNow));
    var post1aTitle = await jinagaClient.Fact(new PostTitle(post1a, "It Depends: Call Your Assisticant", new PostTitle[0]));
    var post1b = await jinagaClient.Fact(new Post(siteFacts[0], user, DateTime.UtcNow.AddMilliseconds(1)));
    var post1bTitle = await jinagaClient.Fact(new PostTitle(post1b, "Information or Disorder?", new PostTitle[0]));

    var post2a = await jinagaClient.Fact(new Post(siteFacts[1], user, DateTime.UtcNow));
    var post2aTitle = await jinagaClient.Fact(new PostTitle(post2a, "Four Theorems for Every Developer", new PostTitle[0]));
    var post2b = await jinagaClient.Fact(new Post(siteFacts[1], user, DateTime.UtcNow.AddMilliseconds(1)));
    var post2bTitle = await jinagaClient.Fact(new PostTitle(post2b, "Let's Write a Topological Sort", new PostTitle[0]));
}